In [ ]:
# -------------------------경로 재설정 및 필수 모듈 재설치----------------------------
# 경로 지정 (필요 시 드라이브 마운트 제외)
from google.colab import drive
drive.mount('/content/drive')
#bi_lstm_attention_tokenizer.pickle
TOKENPATH_ABL = '/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle'
#bi_lstm_attention_model.h5
MODELPATH_ABL = "/content/drive/MyDrive/finalvoice/bi_lstm_attention_model.h5"
#/kobert_model
model_path_KBT = "/content/drive/MyDrive/finalvoice/kobert_model"
#/koelectra_saved_model
model_path_KER = '/content/drive/MyDrive/finalvoice/koelectra_saved_model'
# 라이브러리 설치 (필수)
!pip install konlpy
!pip install accelerate>=0.20.1 transformers[torch]
'''
!pip install transformers
!pip install accelerate>=0.20.1
!pip install transformers[torch]
!pip install accelerate transformers[torch]
'''
# ---------------------------------Attention+Bi-LSTM Model Setting---------------------------------
import pandas as pd
# 커스텀레이어 정의
from tensorflow.keras.layers import Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)
    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
# 필요 모듈, tokenizer, 모델 불러오기
from konlpy.tag import Okt
okt = Okt()
import pickle
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
abl_model = load_model(MODELPATH_ABL, custom_objects={'AttentionLayer': AttentionLayer})
with open(TOKENPATH_ABL, 'rb') as handle:
    keras_tokenizer = pickle.load(handle)
# 확률연산
def predict_label_prob_ABL(sentence):
    # 문장 토큰화
    tokenized_sentence = okt.morphs(sentence)
    # 토큰화된 문장을 숫자 시퀀스로 변환 (Keras의 Tokenizer 사용)
    sequence = keras_tokenizer.texts_to_sequences([tokenized_sentence])
    # 패딩 처리
    MAX_LENGTH = 100
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH)
    # 예측 수행
    prediction_ABL = abl_model.predict(padded_sequence)
    # 예측 확률 반환
    return prediction_ABL[0][0]
'''
user_input = input("문장을 입력하세요: ")
probs_ABL = predict_label_prob_ABL(user_input)
print(f"라벨 0의 확률: {1 - probs_ABL:.4f}")
print(f"라벨 1의 확률: {probs_ABL:.4f}")
'''
# ---------------------------------KoBERT Model Setting---------------------------------
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F
# KoBERT 토크나이저 불러오기
tokenizer_KBT = BertTokenizer.from_pretrained('monologg/kobert')
# 모델 구조 불러오기
model_KBT = BertForSequenceClassification.from_pretrained('monologg/kobert')
# 저장된 가중치 불러오기
model_KBT.load_state_dict(torch.load(model_path_KBT, map_location=torch.device('cpu')))
model_KBT.eval()
def get_prediction_probabilities_KBT(text):
    # 텍스트를 토크나이징
    inputs = tokenizer_KBT.encode_plus(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # 예측
    with torch.no_grad():
        outputs = model_KBT(**inputs)
        logits = outputs.logits
    # 확률을 계산하기 위해 softmax 함수 사용
    probs_KBT = F.softmax(logits, dim=-1)
    # 확률값 반환
    return probs_KBT[0].tolist()
'''
# 사용자 입력 받기
user_input = input("텍스트를 입력하세요: ")
probs_KBT = get_prediction_probabilities_KBT(user_input)
# 각 라벨의 확률 출력
print(f"라벨 0의 확률: {probs_KBT[0]:.4f}")
print(f"라벨 1의 확률: {probs_KBT[1]:.4f}")
'''
# ---------------------------------KoELECRTA Model Setting---------------------------------
from transformers import ElectraForSequenceClassification, ElectraTokenizer
import torch.nn.functional as F
# 1. 저장된 모델과 토크나이저 불러오기
tokenizer_KER = ElectraTokenizer.from_pretrained(model_path_KER)
model = ElectraForSequenceClassification.from_pretrained(model_path_KER)
def get_prediction_probabilities_KER(user_input):
    # 텍스트를 토크나이징
    inputs = tokenizer_KER(user_input, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # 모델을 평가 모드로 설정
    model.eval()
    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    # 확률을 계산하기 위해 softmax 함수 사용
    probs_KER = F.softmax(logits, dim=-1)
    return probs_KER[0].tolist()
'''
# 사용자 입력 받기
user_input = input("텍스트를 입력하세요: ")
probs_KER = get_prediction_probabilities_KER(user_input)
# 각 라벨의 확률 출력
print(f"라벨 0의 확률: {probs_KER[0]:.4f}")
print(f"라벨 1의 확률: {probs_KER[1]:.4f}")
'''
# -----------------------------Trinity(Ensemble) Test------------------------------
while True:
  user_input = input("문장을 입력하세요 (종료하려면 공백 입력): ")
  if not user_input.strip():
    break
  # 각 모델에서 예측 확률을 가져옵니다.
  probs_ABL = predict_label_prob_ABL(user_input)
  probs_KBT = get_prediction_probabilities_KBT(user_input)
  probs_KER = get_prediction_probabilities_KER(user_input)
  # 각 모델에서 예측한 확률을 저장
  bi_lstm_attention_prob_1 = probs_ABL
  bi_lstm_attention_prob_0 = 1 - probs_ABL
  kobert_prob_1 = probs_KBT[1]
  kobert_prob_0 = probs_KBT[0]
  koelectra_prob_1 = probs_KER[1]
  koelectra_prob_0 = probs_KER[0]
  Trinity_1 = (bi_lstm_attention_prob_1 + kobert_prob_1 + koelectra_prob_1) / 3
  Trinity_0 = (bi_lstm_attention_prob_0 + kobert_prob_0 + koelectra_prob_0) / 3
  #앙상블 최종 결과만 볼 경우
  print(f"보이스피싱 가능성: {Trinity_1:.2f}","\n")
  #데이터프레임으로 출력할 경우
'''
  df = pd.DataFrame({
      'Model': ['Attention+Bi-LSTM', 'KoBERT', 'KoELECRTA', 'Trinity(Ensemble)'],
      'Fish': [bi_lstm_attention_prob_1, kobert_prob_1, koelectra_prob_1, Trinity_1],
      'Clear': [bi_lstm_attention_prob_0, kobert_prob_0, koelectra_prob_0, Trinity_0]
  })
  df = df.round(2)
  print(df)
'''